# RFM Analysis

Overview
• Recency measures the number of days since the last purchase, so lower values indicate more recent activity.
• Frequency is the total number of purchases; higher values signal more engaged customers.
• Monetary sums all spending; higher values suggest higher-value customers.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv(r'C:\Users\mgaig\OneDrive\Data - portfolio\Jupyter Lab\RFM Analysis\Online Retail.csv', encoding='ISO-8859-1')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Clean and aggregate the data

In [2]:
df.dropna(subset=['CustomerID'], inplace=True)

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


df['Amount'] = df['Quantity'] * df['UnitPrice']

rfm_df = df.groupby('CustomerID').agg(
    CustomerID=('CustomerID', 'first'),
    LastPurchaseDate=('InvoiceDate', 'max'),
    Frequency=('InvoiceNo', 'nunique'),
    Monetary=('Amount', 'mean')
).reset_index(drop=True)

print(rfm_df.head().to_markdown(index=False, numalign="left", stralign="left"))


print(rfm_df.info())

| CustomerID   | LastPurchaseDate    | Frequency   | Monetary   |
|:-------------|:--------------------|:------------|:-----------|
| 12346        | 2011-01-18 10:17:00 | 2           | 0          |
| 12347        | 2011-12-07 15:52:00 | 7           | 23.6813    |
| 12348        | 2011-09-25 13:13:00 | 4           | 57.9755    |
| 12349        | 2011-11-21 09:51:00 | 1           | 24.076     |
| 12350        | 2011-02-02 16:01:00 | 1           | 19.6706    |
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CustomerID        4372 non-null   float64       
 1   LastPurchaseDate  4372 non-null   datetime64[ns]
 2   Frequency         4372 non-null   int64         
 3   Monetary          4372 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 136.8 KB
None


# Create RFM Data Model
- Calculatea and assign recency scores based on quantiles

In [3]:

rfm_df['Recency'] = (rfm_df['LastPurchaseDate'].max() + pd.to_timedelta('1D')) - rfm_df['LastPurchaseDate']
rfm_df['Recency'] = rfm_df['Recency'].dt.days

quantiles = rfm_df[['Recency', 'Frequency', 'Monetary']].quantile([0.25, 0.5, 0.75])

rfm_df['RecencyScore'] = pd.cut(rfm_df['Recency'], bins=[-1, quantiles['Recency'][0.25], quantiles['Recency'][0.5], quantiles['Recency'][0.75], float('inf')], labels=[4, 3, 2, 1], include_lowest=True)

rfm_df['FrequencyScore'] = pd.cut(rfm_df['Frequency'], bins=[-1, quantiles['Frequency'][0.25], quantiles['Frequency'][0.5], quantiles['Frequency'][0.75], float('inf')], labels=[1, 2, 3, 4], include_lowest=True)
rfm_df['MonetaryScore'] = pd.cut(rfm_df['Monetary'], bins=[-1, quantiles['Monetary'][0.25], quantiles['Monetary'][0.5], quantiles['Monetary'][0.75], float('inf')], labels=[1, 2, 3, 4], include_lowest=True)

# Concatenate scores
rfm_df['RFM_Score'] = rfm_df['RecencyScore'].astype(str) + rfm_df['FrequencyScore'].astype(str) + rfm_df['MonetaryScore'].astype(str)


print(rfm_df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Display data types
print(rfm_df.info())

| CustomerID   | LastPurchaseDate    | Frequency   | Monetary   | Recency   | RecencyScore   | FrequencyScore   | MonetaryScore   | RFM_Score   |
|:-------------|:--------------------|:------------|:-----------|:----------|:---------------|:-----------------|:----------------|:------------|
| 12346        | 2011-01-18 10:17:00 | 2           | 0          | 326       | 1              | 2                | 1               | 121         |
| 12347        | 2011-12-07 15:52:00 | 7           | 23.6813    | 2         | 4              | 4                | 4               | 444         |
| 12348        | 2011-09-25 13:13:00 | 4           | 57.9755    | 75        | 2              | 3                | 4               | 234         |
| 12349        | 2011-11-21 09:51:00 | 1           | 24.076     | 19        | 3              | 1                | 4               | 314         |
| 12350        | 2011-02-02 16:01:00 | 1           | 19.6706    | 310       | 1              | 1                | 3         

# Segment customers using RFM values

In [4]:
def rfm_level(df):
    r = df['RecencyScore']
    f = df['FrequencyScore']
    m = df['MonetaryScore']
    rfm = df['RFM_Score']
    
    
    if r == 4 and f == 4 and m == 4:
        return 'Champions'
    
   
    elif f == 4 and m >= 3:
        return 'Loyal Customers'
    
    
    elif r == 4 and f >= 2 and m >= 1:
        return 'Potential Loyalist'
    
    elif r >= 3 and f >= 3 or r >= 3 and m >= 3:
        return 'Promising'
    
    
    elif r >= 3 and f >= 1 and m >= 1:
        return 'Recent Customers'
    
    
    elif r <= 2 and f >= 1 and m == 4:
        return 'At Risk Big Spenders'
    

    elif r <= 2 and f >= 1 and m >= 3:
        return 'At Risk Moderate Spenders'


    
    elif r >= 2 and f >= 2 and m >= 1:
        return 'Needs Attention'
    

    elif r == 1 and f <= 2 or r == 1 and m <= 2:
        return 'Hibernating'

    
    else:
        return 'About to Sleep'

  
rfm_df['RFM_Level'] = rfm_df.apply(rfm_level, axis=1)


print(rfm_df.head().to_markdown(index=False, numalign="left", stralign="left"))
print(rfm_df.info())


| CustomerID   | LastPurchaseDate    | Frequency   | Monetary   | Recency   | RecencyScore   | FrequencyScore   | MonetaryScore   | RFM_Score   | RFM_Level                 |
|:-------------|:--------------------|:------------|:-----------|:----------|:---------------|:-----------------|:----------------|:------------|:--------------------------|
| 12346        | 2011-01-18 10:17:00 | 2           | 0          | 326       | 1              | 2                | 1               | 121         | Hibernating               |
| 12347        | 2011-12-07 15:52:00 | 7           | 23.6813    | 2         | 4              | 4                | 4               | 444         | Champions                 |
| 12348        | 2011-09-25 13:13:00 | 4           | 57.9755    | 75        | 2              | 3                | 4               | 234         | At Risk Big Spenders      |
| 12349        | 2011-11-21 09:51:00 | 1           | 24.076     | 19        | 3              | 1                | 4               

## Display the number of customers in each RFM Level

In [5]:
# Calculate the number of customers in each RFM level
rfm_level_counts = rfm_df['RFM_Level'].value_counts().reset_index()
rfm_level_counts.columns = ['RFM_Level', 'count']

# Display the result
print(rfm_level_counts.to_markdown(index=False, numalign="left", stralign="left"))

# Display total counts
total_counts = rfm_level_counts['count'].sum()
print(f"Total number of customers: {total_counts}")

| RFM_Level                 | count   |
|:--------------------------|:--------|
| Potential Loyalist        | 690     |
| Promising                 | 628     |
| Hibernating               | 572     |
| At Risk Big Spenders      | 526     |
| At Risk Moderate Spenders | 493     |
| Loyal Customers           | 418     |
| Recent Customers          | 379     |
| Needs Attention           | 322     |
| Champions                 | 186     |
| About to Sleep            | 158     |
Total number of customers: 4372


## Display chart of number of customers in each RFM Level

In [6]:
import altair as alt

# Create a bar chart of number of customers in each `RFM_Level`
chart = alt.Chart(rfm_df).mark_bar().encode(
    x=alt.X('RFM_Level'),
    y=alt.Y('count()', title='Number of Customers'),
    color=alt.Color('RFM_Level'),
    tooltip=['RFM_Level', alt.Tooltip('count()', title='Number of Customers')]
).properties(
    title='Number of Customers per RFM Level'
).interactive()

# Save the chart
chart.save('rfm_level_bar_chart.json')

# Display the chart
chart.display()

alt.Chart(...)

## Output the RFM Data Model to a CSV file

In [7]:
output_path = r'C:\Users\mgaig\OneDrive\Data - portfolio\Jupyter Lab\RFM Analysis\output\RFM_Analysis.csv'
rfm_df.to_csv(output_path, index=False)

# Merge the RFM Levels with the raw order data for time series analysis

In [ ]:

df = df.merge(rfm_df[['CustomerID', 'RFM_Level']], on='CustomerID', how='left')


print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

output_path = r'C:\Users\mgaig\OneDrive\Data - portfolio\Jupyter Lab\RFM Analysis\output\RFM_Analysis_all_sales.csv'
df.to_csv(output_path, index=False)

| InvoiceNo   | StockCode   | Description                         | Quantity   | InvoiceDate         | UnitPrice   | CustomerID   | Country        | Amount   | RFM_Level       |
|:------------|:------------|:------------------------------------|:-----------|:--------------------|:------------|:-------------|:---------------|:---------|:----------------|
| 536365      | 85123A      | WHITE HANGING HEART T-LIGHT HOLDER  | 6          | 2010-12-01 08:26:00 | 2.55        | 17850        | United Kingdom | 15.3     | Loyal Customers |
| 536365      | 71053       | WHITE METAL LANTERN                 | 6          | 2010-12-01 08:26:00 | 3.39        | 17850        | United Kingdom | 20.34    | Loyal Customers |
| 536365      | 84406B      | CREAM CUPID HEARTS COAT HANGER      | 8          | 2010-12-01 08:26:00 | 2.75        | 17850        | United Kingdom | 22       | Loyal Customers |
| 536365      | 84029G      | KNITTED UNION FLAG HOT WATER BOTTLE | 6          | 2010-12-01 08:26:00 | 3.39   